<a href="https://colab.research.google.com/github/smgyl/246/blob/main/hazard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np


import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, LogisticRegressionCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, auc, roc_curve, plot_roc_curve
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter


Mounted at /content/drive


In [2]:
import os
os.chdir('drive/Shareddrives/MS&E 246')


In [22]:
data = pd.read_csv("PreprocessedDset2.csv",low_memory = False)
data2 = pd.read_csv("CHGOFFset.csv",low_memory = False)
print(data.columns)
final_data = data

Index(['BorrState', 'GrossApproval', 'ApprovalDate', 'ApprovalFiscalYear',
       'TermInMonths', 'ProjectState', 'ChargeOffDate', 'GrossChargeOffAmount',
       'GDPBS', 'perPIBS', 'GDPPS', 'perPIPS', 'ApprovalDate_SNP',
       'ApprovalDate_LogSNP', 'BorrStateUnem', 'ProjStateUnem', 'logGDPBS',
       'logGDPPS', 'ChargeOffYear', 'BorrStateUnem_CharOffYr',
       'ProjStateUnem_CharOffYr', 'IfRepBorr', 'BorrState=ProjState',
       'BorrState=CDCState', 'IfChargeOff', 'ThirdPartyDollar_PCT', 'NC_11',
       'NC_21', 'NC_22', 'NC_23', 'NC_31', 'NC_32', 'NC_33', 'NC_42', 'NC_44',
       'NC_45', 'NC_48', 'NC_49', 'NC_51', 'NC_52', 'NC_53', 'NC_54', 'NC_55',
       'NC_56', 'NC_61', 'NC_62', 'NC_71', 'NC_72', 'NC_81', 'NC_92', 'NC_99',
       'NC_na', 'DelMeth_504', 'DelMeth_504REFI', 'DelMeth_ALP',
       'DelMeth_PCLP', 'BusType_CORPORATION', 'BusType_INDIVIDUAL',
       'BusType_PARTNERSHIP', 'TermIsYear', 'log_GrossApproval',
       'norm_logGrossAppv', 'norm_TermInMonths', 'norm_lo

In [23]:
data.head()



,BorrState,GrossApproval,ApprovalDate,ApprovalFiscalYear,TermInMonths,ProjectState,ChargeOffDate,GrossChargeOffAmount,GDPBS,perPIBS,GDPPS,perPIPS,ApprovalDate_SNP,ApprovalDate_LogSNP,BorrStateUnem,ProjStateUnem,logGDPBS,logGDPPS,ChargeOffYear,BorrStateUnem_CharOffYr,ProjStateUnem_CharOffYr,IfRepBorr,BorrState=ProjState,BorrState=CDCState,IfChargeOff,ThirdPartyDollar_PCT,NC_11,NC_21,NC_22,NC_23,NC_31,NC_32,NC_33,NC_42,NC_44,NC_45,NC_48,NC_49,NC_51,NC_52,NC_53,NC_54,NC_55,NC_56,NC_61,NC_62,NC_71,NC_72,NC_81,NC_92,NC_99,NC_na,DelMeth_504,DelMeth_504REFI,DelMeth_ALP,DelMeth_PCLP,BusType_CORPORATION,BusType_INDIVIDUAL,BusType_PARTNERSHIP,TermIsYear,log_GrossApproval,norm_logGrossAppv,norm_TermInMonths,norm_logSNP,norm_BorrStateUnem,norm_ProjStateUnem,norm_BorrStateUnem_COY,norm_ProjStateUnem_COY,norm_GDPBS,norm_perPIBS,norm_GDPPS,norm_perPIPS,norm_ApprovalDate_SNP,norm_logGDPBS,norm_logGDPPS,TimeDiff,ChargeOff_PCT,OneDefault,FiveDefault,Chargeoff_0
0,KS,166000,1990-01-02,1990,12,KS,NaN,0.0,50234.236332,18438.0,50234.236332,18438.0,359.690002,5.885243,4.3,4.3,10.824452,10.824452,NaN,0.0,0.0,0,1,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,12.019743,-0.712909,-8.546375,-2.918255,-0.844068,-0.843982,-0.244431,-0.244435,-0.918611,-1.914168,-0.917919,-1.913181,-2.323081,-1.684921,-1.683642,NaN,0.0,0,0,1
1,CA,117000,1990-01-02,1990,240,CA,NaN,0.0,730332.945533,21485.0,730332.945533,21485.0,359.690002,5.885243,5.8,5.8,13.501256,13.501256,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,11.669929,-0.811549,0.217618,-2.918255,-0.139127,-0.139040,-0.244431,-0.244435,0.178946,-1.565004,0.179678,-1.564115,-2.323081,0.631585,0.632469,NaN,0.0,0,0,1
2,IN,261000,1990-01-03,1990,120,IN,NaN,0.0,123386.189860,17768.0,123386.189860,17768.0,358.760010,5.882654,5.3,5.3,11.723074,11.723074,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,12.472276,-0.521667,-4.395010,-2.925263,-0.374107,-0.374021,-0.244431,-0.244435,-0.800557,-1.990946,-0.799861,-1.989937,-2.325923,-0.907253,-0.906107,NaN,0.0,0,0,1
3,IL,262000,1990-01-03,1990,240,IL,2003-03-28,0.0,299862.524906,21025.0,299862.524906,21025.0,358.760010,5.882654,6.1,6.1,12.611079,12.611079,2003.0,6.8,6.8,0,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,12.476100,-0.519654,0.217618,-2.925263,0.001862,0.001948,3.173411,3.173182,-0.515756,-1.617716,-0.515049,-1.616813,-2.325923,-0.138774,-0.137758,13.238356,0.0,0,0,1
4,UT,154000,1990-01-03,1990,240,UT,NaN,0.0,35267.817137,15010.0,35267.817137,15010.0,358.760010,5.882654,4.4,4.4,10.470726,10.470726,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,11.944708,-0.737065,0.217618,-2.925263,-0.797072,-0.796986,-0.244431,-0.244435,-0.942765,-2.306993,-0.942073,-2.305895,-2.325923,-1.991035,-1.989704,NaN,0.0,0,0,1


In [24]:
data['TimeDiff'] = data['TimeDiff'].replace(np.nan, 0)
data['duration'] = data['TimeDiff'] * data['IfChargeOff'] + data['TermInMonths'] * (1 - data['IfChargeOff']) / 12
data.head()

,BorrState,GrossApproval,ApprovalDate,ApprovalFiscalYear,TermInMonths,ProjectState,ChargeOffDate,GrossChargeOffAmount,GDPBS,perPIBS,GDPPS,perPIPS,ApprovalDate_SNP,ApprovalDate_LogSNP,BorrStateUnem,ProjStateUnem,logGDPBS,logGDPPS,ChargeOffYear,BorrStateUnem_CharOffYr,ProjStateUnem_CharOffYr,IfRepBorr,BorrState=ProjState,BorrState=CDCState,IfChargeOff,ThirdPartyDollar_PCT,NC_11,NC_21,NC_22,NC_23,NC_31,NC_32,NC_33,NC_42,NC_44,NC_45,NC_48,NC_49,NC_51,NC_52,...,NC_54,NC_55,NC_56,NC_61,NC_62,NC_71,NC_72,NC_81,NC_92,NC_99,NC_na,DelMeth_504,DelMeth_504REFI,DelMeth_ALP,DelMeth_PCLP,BusType_CORPORATION,BusType_INDIVIDUAL,BusType_PARTNERSHIP,TermIsYear,log_GrossApproval,norm_logGrossAppv,norm_TermInMonths,norm_logSNP,norm_BorrStateUnem,norm_ProjStateUnem,norm_BorrStateUnem_COY,norm_ProjStateUnem_COY,norm_GDPBS,norm_perPIBS,norm_GDPPS,norm_perPIPS,norm_ApprovalDate_SNP,norm_logGDPBS,norm_logGDPPS,TimeDiff,ChargeOff_PCT,OneDefault,FiveDefault,Chargeoff_0,duration
0,KS,166000,1990-01-02,1990,12,KS,NaN,0.0,50234.236332,18438.0,50234.236332,18438.0,359.690002,5.885243,4.3,4.3,10.824452,10.824452,NaN,0.0,0.0,0,1,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,12.019743,-0.712909,-8.546375,-2.918255,-0.844068,-0.843982,-0.244431,-0.244435,-0.918611,-1.914168,-0.917919,-1.913181,-2.323081,-1.684921,-1.683642,0.000000,0.0,0,0,1,1.000000
1,CA,117000,1990-01-02,1990,240,CA,NaN,0.0,730332.945533,21485.0,730332.945533,21485.0,359.690002,5.885243,5.8,5.8,13.501256,13.501256,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,11.669929,-0.811549,0.217618,-2.918255,-0.139127,-0.139040,-0.244431,-0.244435,0.178946,-1.565004,0.179678,-1.564115,-2.323081,0.631585,0.632469,0.000000,0.0,0,0,1,20.000000
2,IN,261000,1990-01-03,1990,120,IN,NaN,0.0,123386.189860,17768.0,123386.189860,17768.0,358.760010,5.882654,5.3,5.3,11.723074,11.723074,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,12.472276,-0.521667,-4.395010,-2.925263,-0.374107,-0.374021,-0.244431,-0.244435,-0.800557,-1.990946,-0.799861,-1.989937,-2.325923,-0.907253,-0.906107,0.000000,0.0,0,0,1,10.000000
3,IL,262000,1990-01-03,1990,240,IL,2003-03-28,0.0,299862.524906,21025.0,299862.524906,21025.0,358.760010,5.882654,6.1,6.1,12.611079,12.611079,2003.0,6.8,6.8,0,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,12.476100,-0.519654,0.217618,-2.925263,0.001862,0.001948,3.173411,3.173182,-0.515756,-1.617716,-0.515049,-1.616813,-2.325923,-0.138774,-0.137758,13.238356,0.0,0,0,1,13.238356
4,UT,154000,1990-01-03,1990,240,UT,NaN,0.0,35267.817137,15010.0,35267.817137,15010.0,358.760010,5.882654,4.4,4.4,10.470726,10.470726,NaN,0.0,0.0,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,1,11.944708,-0.737065,0.217618,-2.925263,-0.797072,-0.796986,-0.244431,-0.244435,-0.942765,-2.306993,-0.942073,-2.305895,-2.325923,-1.991035,-1.989704,0.000000,0.0,0,0,1,20.000000


In [25]:
from lifelines import CoxPHFitter


In [68]:
fdata = data.drop(labels=['DelMeth_PCLP','NC_na','GDPBS', 'perPIBS', 'GDPPS', 'perPIPS', 'ApprovalDate_SNP',
       'ApprovalDate_LogSNP', 'BorrStateUnem', 'ProjStateUnem', 'logGDPBS',
       'logGDPPS', 'BorrStateUnem_CharOffYr', 'ProjStateUnem_CharOffYr','GrossApproval', 'ApprovalDate', 'ApprovalFiscalYear',
       'TermInMonths', 'ProjectState', 'ChargeOffDate', 'GrossChargeOffAmount','TimeDiff', 'ChargeOff_PCT', 'OneDefault',
       'FiveDefault', 'Chargeoff_0', 'ChargeOffYear', 'BorrState',   
     'BusType_PARTNERSHIP'], axis=1)
print(fdata.columns)
fdata.head()

Index(['IfRepBorr', 'BorrState=ProjState', 'BorrState=CDCState', 'IfChargeOff',
       'ThirdPartyDollar_PCT', 'NC_11', 'NC_21', 'NC_22', 'NC_23', 'NC_31',
       'NC_32', 'NC_33', 'NC_42', 'NC_44', 'NC_45', 'NC_48', 'NC_49', 'NC_51',
       'NC_52', 'NC_53', 'NC_54', 'NC_55', 'NC_56', 'NC_61', 'NC_62', 'NC_71',
       'NC_72', 'NC_81', 'NC_92', 'NC_99', 'DelMeth_504', 'DelMeth_504REFI',
       'DelMeth_ALP', 'BusType_CORPORATION', 'BusType_INDIVIDUAL',
       'TermIsYear', 'log_GrossApproval', 'norm_logGrossAppv',
       'norm_TermInMonths', 'norm_logSNP', 'norm_BorrStateUnem',
       'norm_ProjStateUnem', 'norm_BorrStateUnem_COY',
       'norm_ProjStateUnem_COY', 'norm_GDPBS', 'norm_perPIBS', 'norm_GDPPS',
       'norm_perPIPS', 'norm_ApprovalDate_SNP', 'norm_logGDPBS',
       'norm_logGDPPS', 'duration'],
      dtype='object')


,IfRepBorr,BorrState=ProjState,BorrState=CDCState,IfChargeOff,ThirdPartyDollar_PCT,NC_11,NC_21,NC_22,NC_23,NC_31,NC_32,NC_33,NC_42,NC_44,NC_45,NC_48,NC_49,NC_51,NC_52,NC_53,NC_54,NC_55,NC_56,NC_61,NC_62,NC_71,NC_72,NC_81,NC_92,NC_99,DelMeth_504,DelMeth_504REFI,DelMeth_ALP,BusType_CORPORATION,BusType_INDIVIDUAL,TermIsYear,log_GrossApproval,norm_logGrossAppv,norm_TermInMonths,norm_logSNP,norm_BorrStateUnem,norm_ProjStateUnem,norm_BorrStateUnem_COY,norm_ProjStateUnem_COY,norm_GDPBS,norm_perPIBS,norm_GDPPS,norm_perPIPS,norm_ApprovalDate_SNP,norm_logGDPBS,norm_logGDPPS,duration
0,0,1,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,12.019743,-0.712909,-8.546375,-2.918255,-0.844068,-0.843982,-0.244431,-0.244435,-0.918611,-1.914168,-0.917919,-1.913181,-2.323081,-1.684921,-1.683642,1.000000
1,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,11.669929,-0.811549,0.217618,-2.918255,-0.139127,-0.139040,-0.244431,-0.244435,0.178946,-1.565004,0.179678,-1.564115,-2.323081,0.631585,0.632469,20.000000
2,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,12.472276,-0.521667,-4.395010,-2.925263,-0.374107,-0.374021,-0.244431,-0.244435,-0.800557,-1.990946,-0.799861,-1.989937,-2.325923,-0.907253,-0.906107,10.000000
3,0,1,1,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,12.476100,-0.519654,0.217618,-2.925263,0.001862,0.001948,3.173411,3.173182,-0.515756,-1.617716,-0.515049,-1.616813,-2.325923,-0.138774,-0.137758,13.238356
4,0,1,1,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,11.944708,-0.737065,0.217618,-2.925263,-0.797072,-0.796986,-0.244431,-0.244435,-0.942765,-2.306993,-0.942073,-2.305895,-2.325923,-1.991035,-1.989704,20.000000


In [69]:
cph = CoxPHFitter()
cph.fit(fdata, duration_col='duration', event_col='IfChargeOff')

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 54052 total observations, 45246 right-censored observations>
             duration col = 'duration'
                event col = 'IfChargeOff'
      baseline estimation = breslow
   number of observations = 54052
number of events observed = 8806
   partial log-likelihood = -75797.53
         time fit was run = 2021-03-10 01:29:53 UTC

---
                         coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                      
IfRepBorr               -0.04       0.96       0.11            -0.25             0.17                 0.78                 1.18
BorrState=ProjState      0.10       1.11       0.22            -0.34             0.54                 0.71                 1.72
BorrState=CDCState       0.09       1.10       0.04             0.01             0.17                 1.01                 1.19
ThirdPartyDollar_PCT     0.02       1.02       0.00             0.02             0.03                 1.02                 1.03
NC_11                    0.75       2.11       0.15             0.46             1.04                 1.59                 2.82
NC_21                    1.02       2.77       0.36             0.32             1.72                 1.38                 5.57
NC_22                    0.32       1.38       1.00            -1.64             2.29                 0.19                 9.84
NC_23                    0.17       1.19       0.06             0.06             0.28                 1.06                 1.33
NC_31                    0.39       1.47       0.10             0.20             0.57                 1.22                 1.77
NC_32                    0.30       1.34       0.07             0.16             0.43                 1.18                 1.53
NC_33                    0.15       1.16       0.06             0.03             0.26                 1.03                 1.30
NC_42                    0.15       1.16       0.06             0.02             0.27                 1.02                 1.31
NC_44                    0.35       1.43       0.05             0.26             0.45                 1.29                 1.57
NC_45                    0.20       1.22       0.09             0.03             0.37                 1.03                 1.45
NC_48                    0.35       1.42       0.12             0.12             0.58                 1.13                 1.79
NC_49                    0.30       1.35       0.25            -0.19             0.80                 0.82                 2.22
NC_51                    0.55       1.73       0.13             0.29             0.80                 1.34                 2.22
NC_52                    0.16       1.18       0.08             0.00             0.33                 1.00                 1.39
NC_53                    0.19       1.20       0.08             0.03             0.34                 1.04                 1.40
NC_54                   -0.06       0.94       0.06            -0.18             0.06                 0.83                 1.07
NC_55                    0.24       1.27       0.58            -0.90             1.38                 0.41                 3.96
NC_56                   -0.11       0.90       0.10            -0.31             0.09                 0.74                 1.10
NC_61                    0.17       1.19       0.13            -0.09             0.43                 0.92                 1.54
NC_62                   -0.04       0.96       0.06            -0.16             0.07                 0.85                 1.07
NC_71                    0.42       1.52       0.07             0.28             0.55                 1.33                 1.73
NC_72                    0.48       1.62       0.05             0.39             0.57                 1.48                 1.77
NC_81                    0.25       1.29